In [1]:
%matplotlib inline
from ipywidgets import widgets, interact, BoundedFloatText, FloatSlider, Layout, SliderStyle, Button
import numpy as np
import matplotlib.pyplot as plt
from ctrl_sim import *
import time
from IPython.display import HTML, display
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation

class Visualizer:
    def __init__(self):
        # 力学系のパラメータ入力テキストボックス
        self.M_input = BoundedFloatText(value=1.0, min=0.01, max=10.0, step=0.01, description='台車質量:', disabled=False)
        self.m_input = BoundedFloatText(value=1.0, min=0.01, max=10.0, step=0.01, description='振子質量', disabled=False)
        self.l_input = BoundedFloatText(value=0.5, min=0.01, max=10.0, step=0.01, description='振子長:', disabled=False)
        self.M = 1.0
        self.m = 1.0
        self.l = 0.5
        
        # パラメータ変更時のイベントハンドラを登録
        self.M_input.observe(self.set_model_param, names='value')
        self.m_input.observe(self.set_model_param, names='value')
        self.l_input.observe(self.set_model_param, names='value')
        
        # アニメーション実行ボタン(重い >_<;)
        self.ani_button = Button(
            description="アニメーション",
            button_style="info",
            icon="play"
        )
        self.ani_button.on_click(self.on_ani_button)

        # アニメーション保存ボタン(重い (     ´・ω・`    ;;;;))
        self.sav_button = Button(
            description="保存",
            button_style="info",
            icon="save"
        )
        self.sav_button.on_click(self.on_sav_button)

        # 結果の表示領域
        self.plot_area = widgets.Output() # グラフ表示領域
        self.ani_area = widgets.Output() # アニメーション表示領域
        display(self.plot_area, self.ani_area)

        # テキストボックスを表示
        display(widgets.HBox([self.M_input, self.m_input, self.l_input]))
        
        # PIDパラメータを設定するスライダーを作成
        interact(self.on_slide,
                 x_kp=FloatSlider(min=0.0, max=200.0, step=0.01, value=80.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')),
                 x_ki=FloatSlider(min=0.0, max=200.0, step=0.01, value=0.4, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')), 
                 x_kd=FloatSlider(min=0.0, max=200.0, step=0.01, value=1.2, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')),
                 theta_kp=FloatSlider(min=0.0, max=200.0, step=0.01, value=120.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')),
                 theta_ki=FloatSlider(min=0.0, max=200.0, step=0.01, value=0.6, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')),
                 theta_kd=FloatSlider(min=0.0, max=200.0, step=0.01, value=1.8, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue'))
                )
        # ボタンを表示
        display(widgets.HBox([self.ani_button, self.sav_button]))        
    
    # シミュレーション実行メソッド
    def simulate(self, param):
        M, m, l, kp, ki, kd = param
        sys = Inverted_Pendulum_System(M, m, l)
        ctrl = PID_Controller(2, 1)
        dms = Simulator(sys, ctrl)

        self.sys = sys
        self.ctrl = ctrl
        self.dms = dms
    
        maxT = 5.0
        dt = 0.01
        self.dt = dt
        sys.set_observer(lambda x: x[:2])
        
        # x, theta, x_dot, theta_dot
        dms.set_aim([0.0, np.pi, 0.0, 0.0], [0.0, 0.0], [0.0])
        dms.ctrl.set_pid(kp, ki, kd)
        principal_arg = lambda theta: theta - np.floor(theta / (2*np.pi)) * 2*np.pi
        f_err = lambda x, x_ref: casadi.DM([x_ref[0] - x[0], -principal_arg(x_ref[1] - x[1])])
        dms.ctrl.set_error_func(f_err)
        
        t_start = time.perf_counter()
        self.is_attained = dms.execute_until_stationary(maxT=maxT, dt=dt, threshold=0.1)
        self.time = [v[0] for v in sys.history]
        t_control = time.perf_counter()
        with self.plot_area:
            self.plot_result(dms)
        t_plot = time.perf_counter()
        
        print(f'control calc time: {t_control - t_start:.3f}sec; plot time: {t_plot - t_control:.3f}sec')

    def draw_pendulum(self, ax, x, theta, u):
        l = self.l
        u_scale = 15
        points = np.array([
            [x,x-l*np.sin(theta)],
            [0,l*np.cos(theta)]
        ])
        ax.scatter(*points,color="blue", s=50)
        ax.plot(*points, color='blue', lw=2)
        if u:
            ax.arrow(x,0,u/u_scale,0,width=0.02,head_width=0.06,head_length=0.12,length_includes_head=False,color="green",zorder=3)
    
        w = 0.2
        h = 0.1
        rect = patches.Rectangle(xy=(x-w/2,-h/2), width=w, height=h,color="black")
        ax.add_patch(rect)

    def update_figure(self, i):
        x_lim_min = -12
        x_lim_max = 3
        y_lim_min = -2
        y_lim_max = 2
        
        ax = self.ax
    
        ax.cla()
        ax.set_xlim(x_lim_min, x_lim_max)
        ax.set_ylim(y_lim_min, y_lim_max)
        ax.set_aspect("equal")
        ax.set_title(f"t={self.time[i]:0.2f}")
        
        ax.hlines(0,x_lim_min,x_lim_max,colors="black")
        x,theta,_,_ = self.sys.history[i][1]
        u = self.sys.history[i][2][0]
        self.draw_pendulum(ax, x, theta, u)

    def animation(self):
        self.ani_fig = plt.figure(figsize=(10, 5))
        self.ax = self.ani_fig.add_subplot(111)
        self.frames = np.arange(0, len(self.time))
        self.ani = FuncAnimation(self.ani_fig, self.update_figure, frames=self.frames)
        self.ani_html = HTML(self.ani.to_jshtml())
        display(self.ani_html)

    def plot_result(self, dms):
        fig = plt.figure(figsize=(12, 6))

        M = dms.sys.param.get('M')
        m = dms.sys.param.get('m')
        l = dms.sys.param.get('l')
        kp = dms.ctrl.get_param('kp')[0,0]
        ki = dms.ctrl.get_param('ki')[0,0]
        kd = dms.ctrl.get_param('kd')[0,0]

        title_str = 'Inverted Pendulum PID Simulation: '
        if self.is_attained:
            title_str += 'Attained in %.3f sec\n' % (dms.sys.t)
        else:
            title_str += 'not attained\n'
        title_str += '(M, m, l) = (%.1f, %.1f, %.1f)' % (M, m, l)
#        title_str += '(kp, ki, kd) = (%.3f, %.3f, %.3f)' % (kp, ki, kd)
        fig.suptitle(title_str)

        for i in range(4):
            ax = fig.add_subplot(3, 2, 1+i)
            self.sys.plot_state(ax, [i])
            ax.legend()
            
        ax = fig.add_subplot(3, 2, 5)
        self.sys.plot_control(ax)
        ax.legend()

        ax = fig.add_subplot(3, 2, 6)
        x, theta, _, _ = self.sys.x.full()[:, 0]
        self.draw_pendulum(ax, x, theta, 0.0)

    # 力学系のパラメータが変更されたら更新する
    def set_model_param(self, change):
        self.M = self.M_input.value
        self.m = self.m_input.value
        self.l = self.l_input.value

        self.bug = 1/0

        self.exec_sim()

    # PIDパラメータが変更されたら更新する
    def on_slide(self, x_kp, x_ki, x_kd, theta_kp, theta_ki, theta_kd):
        self.x_kp = x_kp
        self.x_ki = x_ki
        self.x_kd = x_kd
        self.theta_kp = theta_kp
        self.theta_ki = theta_ki
        self.theta_kd = theta_kd

        self.exec_sim()

    # シミュレーションを実行
    def exec_sim(self):
        self.simulate((self.M, self.m, self.l, np.array([[self.x_kp, self.theta_kp]]), np.array([[self.x_ki, self.theta_ki]]), np.array([[self.x_kd, self.theta_kd]])))

    # アニメーション作成・表示
    def on_ani_button(self, event):
        with self.ani_area:
            self.ani_area.clear_output(wait=True)
            self.animation()

    # アニメーションをgifとして保存
    def on_sav_button(self, event):
        fps = 1 / self.dt
        self.ani.save("inv_pendulum_PID.gif",writer="pillow",fps=fps)
    
vis = Visualizer()

Output()

Output()

interactive(children=(FloatSlider(value=80.0, description='x_kp', layout=Layout(width='100%'), max=200.0, step…